In [2]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [3]:
df = pd.read_sql("count_contentieux_par_compte", index_col=0)
df.dropna(axis=1, how='all', inplace=True)

In [4]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy="median")
df_numeric = df._get_numeric_data()
print(df_numeric.shape)
imp.fit(df_numeric)
idf = pd.DataFrame(imp.transform(df_numeric))
print(idf.shape)
idf.columns=df_numeric.columns
idf.index=df_numeric.index
df.update(idf)
df._get_numeric_data().isna().sum()/len(df._get_numeric_data())

(100000, 46)
(100000, 46)


nb_ctx                           0.0
nb_comptactifs_std               0.0
nb_comptes_factures_std          0.0
nb_comptes_recouvres_std         0.0
nb_debiteurs_std                 0.0
nb_crediteurs_std                0.0
nb_a_zero_std                    0.0
mnt_ttc_facture_ccl_std          0.0
mnt_tva_facture_ccl_std          0.0
mnt_ttc_echeance_std             0.0
mnt_tva_echeance_std             0.0
mnt_loyer_ccl_std                0.0
mnt_charges_ccl_std              0.0
mnt_caution_dg_ccl_std           0.0
mnt_aides_ccl_std                0.0
mnt_sls_ccl_std                  0.0
mnt_solde_charges_ccl_std        0.0
mnt_autres_ccl_std               0.0
mnt_total_encaisse_std           0.0
solde_std                        0.0
solde_crediteur_std              0.0
solde_debiteur_std               0.0
dette_du_mois_std                0.0
dette_moins_3mois_std            0.0
dette_plus_3_a_6_mois_std        0.0
dette_plus_6_a_12_mois_std       0.0
dette_plus_12_mois_std           0.0
n

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [8]:
df["existence_ctx"] = (df.nb_ctx>0).astype('int64', copy=False)
features = [c for c in df.columns if c not in {"nb_ctx", "existence_ctx"} and '_id' not in c]
X = df[features]._get_numeric_data()


y=df["existence_ctx"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [9]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

<!-- http://127.0.0.1:7887/139716721714904/ -->

In [12]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=None)

lr = LogisticRegression(random_state=None, feature_names=feature_names, penalty='l1', solver='liblinear')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

In [13]:
from interpret.perf import ROC

lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)

<!-- http://127.0.0.1:7887/139716343904800/ -->

<!-- http://127.0.0.1:7887/139716343904240/ -->

In [14]:
lr_global = lr.explain_global(name='Logistic Regression')
tree_global = tree.explain_global(name='Classification Tree')

show(lr_global)
show(tree_global)

<!-- http://127.0.0.1:7887/139716314661104/ -->

<!-- http://127.0.0.1:7887/139716314662056/ -->